In [48]:
TOKENS = "train_tokens.txt"
LABELS = "train_nes.txt"

In [49]:
import re

regex = re.compile('[^а-яА-Я0-9\-a-zA-Z"]')
#First parameter is the replacement, second parameter is your input string
regex.sub('', '"a-b"')
#Out: 'abdE'

'"a-b"'

In [212]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

from collections import defaultdict

In [215]:
tokens = []
word_to_idx = defaultdict(lambda: 0)
with open(TOKENS, 'r') as fin:
    for line in fin:
        
        for word in line.split(" "):
            stemmed = stemmer.stem(word.lower())
            if stemmed not in word_to_idx:
                word_to_idx[stemmed] = len(word_to_idx) + 1

        words = []
        word_pos = 0
        for word in line.split(' '):
            words.append(regex.sub('', word).replace('ё', 'e'))
        word_dict = {}
        i = 0
        while i < len(words):
            if i + 2 < len(words) and words[i] and words[i + 1] and words[i + 2]:
                word_v = re.compile('[^а-яА-Я\-]').sub('', words[i + 2])
                if word_v and len(word_v) > 1:
                    word_dict[(words[i], words[i + 1])] = word_v
            i += 3
        tokens.append(word_dict)
            

In [138]:
from os import listdir
from os.path import isfile, join
txtfiles = [f for f in listdir('Collection3') if isfile(join('Collection3', f)) and f.endswith('.txt')]

In [139]:
txtfiles

['1047.txt',
 '289.txt',
 '504.txt',
 '262.txt',
 '276.txt',
 '20_11_12a.txt',
 '510.txt',
 '538.txt',
 '28_11_12g.txt',
 '2002.txt',
 '2016.txt',
 '060.txt',
 '074.txt',
 '048.txt',
 '114.txt',
 '100.txt',
 '128.txt',
 '1127.txt',
 '1133.txt',
 '470.txt',
 'last_42.txt',
 '316.txt',
 '302.txt',
 '464.txt',
 'last_56.txt',
 '458.txt',
 '459.txt',
 '303.txt',
 'last_57.txt',
 '465.txt',
 'shojgu4.txt',
 'last_43.txt',
 '471.txt',
 '317.txt',
 '1132.txt',
 '1126.txt',
 '129.txt',
 '101.txt',
 '115.txt',
 '049.txt',
 '075.txt',
 '061.txt',
 '2017.txt',
 '2003.txt',
 '28_11_12f.txt',
 '539.txt',
 '277.txt',
 '511.txt',
 '505.txt',
 '263.txt',
 '288.txt',
 '1046.txt',
 '1044.txt',
 '1050.txt',
 '513.txt',
 '20_11_12b.txt',
 '275.txt',
 '261.txt',
 '507.txt',
 '249.txt',
 '04_12_12b.txt',
 '2029.txt',
 '2015.txt',
 '088.txt',
 '2001.txt',
 '077.txt',
 'blokhin.txt',
 '063.txt',
 '103.txt',
 '117.txt',
 '533 (!).txt',
 '1130.txt',
 '1124.txt',
 '498.txt',
 '1118.txt',
 'shojgu6.txt',
 '467.tx

In [140]:
tokens[0]

{('0', '11'): 'Выступления',
 ('14', '10'): 'ресторанах',
 ('27', '7'): 'кабаках',
 ('37', '5'): 'между',
 ('43', '7'): 'горячим',
 ('53', '8'): 'десертом',
 ('65', '12'): 'мюзик-холлах',
 ('80', '13'): 'фешенебельных',
 ('94', '6'): 'отелях',
 ('103', '3'): 'для',
 ('107', '7'): 'королей',
 ('115', '7'): 'Густава',
 ('123', '9'): 'Шведского',
 ('134', '8'): 'Альфонса',
 ('143', '10'): 'Испанского',
 ('155', '6'): 'принца',
 ('162', '9'): 'Уэльского',
 ('173', '3'): 'для',
 ('177', '13'): 'Вандербильтов',
 ('193', '10'): 'Ротшильдов'}

In [141]:
from tqdm import notebook

In [142]:
entities = []
with open(LABELS, 'r') as fin:
    for line in fin:
        words = line.split(' ')
        word_dict = {}
        i = 0
        while i < len(words):
            if i + 2 < len(words) and words[i] and words[i + 1] and words[i + 2]:
                word_dict[(words[i], words[i + 1])] = words[i + 2]
            i += 3
        entities.append(word_dict)
            

In [143]:
for file in notebook.tqdm(txtfiles):
    with open("Collection3/" + file, 'r') as fin1:
        with open("Collection3/" + file.replace('.txt', '.ann'), 'r') as fin2:
            toks = {}
            current_index = 0        
            for line in fin1:
                word_pos = 0
                if line != "":
                    words = re.split('[ \t\.]+', line)
                    for word in words:
                        pure_word = regex.sub('', word).replace('ё', 'e')
                        if pure_word:
                            wrd = re.compile('[^а-яА-Яa-zA-Z]').sub('', pure_word)
                            if wrd and len(wrd) > 2:
                                toks[(str(current_index), str(len(pure_word)))] = wrd
                            #else:
                                #print(wrd)
                        current_index += len(word) + 1
            if toks.keys():
                #tokens.append(toks)
                pass
            else:
                print("OUEYFOUYEGWOF")
  
            word_dict = {}
            for line in fin2:
                words = re.split('[ \t\.]+', line)
                #print(words)
                typ = None
                if words[1] == "PER": 
                    typ = "PERSON"
                if words[1] == "ORG":
                    typ = "ORG"
                if typ:
                    cur_ind = int(words[2])
                    for i in range(4, len(words)):
                        pure_word = regex.sub('', words[i]).replace('ё', 'e')
                        if len(pure_word) > 2:
                            word_dict[(str(cur_ind), str(len(pure_word)))] = typ
                        cur_ind += len(words[i]) + 1
            #entities.append(word_dict)

In [144]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [145]:
morph.parse("dog")[0]

Parse(word='dog', tag=OpencorporaTag('LATN'), normal_form='dog', score=1.0, methods_stack=((<LatinAnalyzer>, 'dog'),))

In [146]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

In [147]:
pos_to_num = {
    'ADJF' : 1,
    'ADJS' : 2,
    'COMP' : 3,
    'VERB' : 4,
    'INFN' : 5,
    'PRTF' : 6,
    'PRTS' : 7,
    'GRND' : 8,
    'NUMR' : 9,
    'ADVB' : 10,
    'NPRO' : 11,
    'PRED' : 12,
    'PREP' : 13,
    'CONJ' : 14,
    'PRCL' : 15,
    'INTJ' : 16,
    'NOUN' : 17
}
def get_pos_to_num(pos):
    if pos in pos_to_num.keys():
        return pos_to_num[pos]
    return 0

In [148]:
number_to_num = {
    'sing' : 1,
    'plur' : 2
}
def get_number_to_num(number):
    if number in number_to_num.keys():
        return number_to_num[number]
    return 0

In [149]:
gender_to_num = {
    'masc' : 1,
    'femn' : 2,
    'neut' : 3
}
def get_gender_to_num(gender):
    if gender in gender_to_num.keys():
        return gender_to_num[gender]
    return 0

In [150]:
case_to_num = {
    'nomn' : 1,
    'gent' : 2,
    'datv' : 3,
    'accs' : 4,
    'ablt' : 5,
    'loct' : 6,
    'voct' : 7,
    'gen2' : 8,
    'acc2' : 9,
    'loc2' : 10
}
def get_case_to_num(case):
    if case in case_to_num.keys():
        return case_to_num[case]
    return 0

In [151]:
from sklearn.preprocessing import OneHotEncoder

In [152]:
def build_features(word, position, sent_length):
    word_features = []
    if word.startswith("\""):
        word_features.append(1)
    else:
        word_features.append(0)
    word.replace("\"", '')
    #if word.containts()
    parse = morph.parse(word)[0]
    word_features.append(get_pos_to_num(parse.tag.POS))
    word_features.append(get_number_to_num(parse.tag.number))
    word_features.append(get_gender_to_num(parse.tag.gender))
    word_features.append(get_case_to_num(parse.tag.case))
    if word[0].isupper():
        word_features.append(1)
    else:
        word_features.append(0)
    
    if word.isupper():
        word_features.append(1)
    else:
        word_features.append(0)
    latin = re.compile('[^a-zA-Z]')
    if latin.search(word):
        word_features.append(1)
    else:
        word_features.append(0)
    
    nums = re.compile('[^0-9]')
    if nums.search(word):
        word_features.append(1)
    else:
        word_features.append(0)
    
    if (position == 1):
        word_features.append(1)
    else:
        word_features.append(0)
    
    word_features.append(position / sent_length)
    return word_features
    

In [153]:
tokens[0]

{('0', '11'): 'Выступления',
 ('14', '10'): 'ресторанах',
 ('27', '7'): 'кабаках',
 ('37', '5'): 'между',
 ('43', '7'): 'горячим',
 ('53', '8'): 'десертом',
 ('65', '12'): 'мюзик-холлах',
 ('80', '13'): 'фешенебельных',
 ('94', '6'): 'отелях',
 ('103', '3'): 'для',
 ('107', '7'): 'королей',
 ('115', '7'): 'Густава',
 ('123', '9'): 'Шведского',
 ('134', '8'): 'Альфонса',
 ('143', '10'): 'Испанского',
 ('155', '6'): 'принца',
 ('162', '9'): 'Уэльского',
 ('173', '3'): 'для',
 ('177', '13'): 'Вандербильтов',
 ('193', '10'): 'Ротшильдов'}

In [154]:
entities[0]

{('115', '7'): 'PERSON',
 ('123', '9'): 'PERSON',
 ('134', '8'): 'PERSON',
 ('143', '10'): 'PERSON',
 ('162', '9'): 'PERSON'}

In [191]:
features = []
labels = []
for i in notebook.tqdm(range(len(entities))):
    tokens_line = tokens[i]
    labels_line = entities[i]
    #print(tokens_line)
    #print(labels_line)
    #break
    position = 1
    for token in tokens_line.keys():
        features.append(build_features(tokens_line[token], position, len(tokens_line)))
        position += 1
        if token in labels_line.keys():
            labels.append(labels_line[token])
        else:
            labels.append("OUT")

In [192]:
#enc.fit(features)

In [193]:
#features = enc.transform(features).toarray()

In [194]:
print(len(features))

31289


In [195]:
from catboost import Pool, CatBoostClassifier

In [196]:
cat_features = [0]
train_dataset = Pool(data=features,
                     label=labels)

clf = CatBoostClassifier(iterations=1000,
                           learning_rate=1,
                           depth=2,
                           loss_function='MultiClass')
clf.fit(train_dataset)

0:	learn: 0.3406879	total: 17.7ms	remaining: 17.7s
1:	learn: 0.3116978	total: 26.4ms	remaining: 13.2s
2:	learn: 0.2759170	total: 34.7ms	remaining: 11.5s
3:	learn: 0.2630628	total: 43.2ms	remaining: 10.8s
4:	learn: 0.2576277	total: 51.3ms	remaining: 10.2s
5:	learn: 0.2511608	total: 59.5ms	remaining: 9.85s
6:	learn: 0.2498791	total: 67.6ms	remaining: 9.59s
7:	learn: 0.2448392	total: 75.9ms	remaining: 9.41s
8:	learn: 0.2421427	total: 84.3ms	remaining: 9.28s
9:	learn: 0.2405955	total: 92.4ms	remaining: 9.14s
10:	learn: 0.2374412	total: 101ms	remaining: 9.07s
11:	learn: 0.2367864	total: 109ms	remaining: 8.97s
12:	learn: 0.2361101	total: 117ms	remaining: 8.9s
13:	learn: 0.2349061	total: 126ms	remaining: 8.85s
14:	learn: 0.2333703	total: 134ms	remaining: 8.78s
15:	learn: 0.2322607	total: 144ms	remaining: 8.86s
16:	learn: 0.2317564	total: 152ms	remaining: 8.81s
17:	learn: 0.2306859	total: 161ms	remaining: 8.76s
18:	learn: 0.2304727	total: 169ms	remaining: 8.71s
19:	learn: 0.2303175	total: 177m

171:	learn: 0.2167073	total: 2.12s	remaining: 10.2s
172:	learn: 0.2166420	total: 2.13s	remaining: 10.2s
173:	learn: 0.2166144	total: 2.15s	remaining: 10.2s
174:	learn: 0.2165125	total: 2.15s	remaining: 10.2s
175:	learn: 0.2165113	total: 2.17s	remaining: 10.1s
176:	learn: 0.2164997	total: 2.17s	remaining: 10.1s
177:	learn: 0.2164957	total: 2.19s	remaining: 10.1s
178:	learn: 0.2164776	total: 2.19s	remaining: 10.1s
179:	learn: 0.2164499	total: 2.2s	remaining: 10s
180:	learn: 0.2164340	total: 2.21s	remaining: 10s
181:	learn: 0.2164098	total: 2.22s	remaining: 9.98s
182:	learn: 0.2164089	total: 2.23s	remaining: 9.95s
183:	learn: 0.2162355	total: 2.24s	remaining: 9.93s
184:	learn: 0.2162271	total: 2.25s	remaining: 9.9s
185:	learn: 0.2162241	total: 2.27s	remaining: 9.91s
186:	learn: 0.2161974	total: 2.28s	remaining: 9.92s
187:	learn: 0.2161920	total: 2.3s	remaining: 9.92s
188:	learn: 0.2161892	total: 2.31s	remaining: 9.91s
189:	learn: 0.2161460	total: 2.32s	remaining: 9.91s
190:	learn: 0.21613

351:	learn: 0.2133678	total: 3.94s	remaining: 7.25s
352:	learn: 0.2133563	total: 3.95s	remaining: 7.24s
353:	learn: 0.2133552	total: 3.96s	remaining: 7.22s
354:	learn: 0.2133519	total: 3.97s	remaining: 7.21s
355:	learn: 0.2133429	total: 3.98s	remaining: 7.2s
356:	learn: 0.2133162	total: 3.99s	remaining: 7.18s
357:	learn: 0.2133141	total: 4s	remaining: 7.17s
358:	learn: 0.2132559	total: 4.01s	remaining: 7.15s
359:	learn: 0.2132501	total: 4.01s	remaining: 7.14s
360:	learn: 0.2132461	total: 4.02s	remaining: 7.12s
361:	learn: 0.2132239	total: 4.03s	remaining: 7.11s
362:	learn: 0.2132228	total: 4.04s	remaining: 7.09s
363:	learn: 0.2132208	total: 4.05s	remaining: 7.08s
364:	learn: 0.2132142	total: 4.06s	remaining: 7.06s
365:	learn: 0.2131924	total: 4.07s	remaining: 7.04s
366:	learn: 0.2131178	total: 4.07s	remaining: 7.03s
367:	learn: 0.2130996	total: 4.08s	remaining: 7.01s
368:	learn: 0.2130536	total: 4.09s	remaining: 6.99s
369:	learn: 0.2130475	total: 4.1s	remaining: 6.98s
370:	learn: 0.213

512:	learn: 0.2118077	total: 5.33s	remaining: 5.06s
513:	learn: 0.2118062	total: 5.34s	remaining: 5.05s
514:	learn: 0.2117396	total: 5.35s	remaining: 5.04s
515:	learn: 0.2117390	total: 5.36s	remaining: 5.02s
516:	learn: 0.2117352	total: 5.36s	remaining: 5.01s
517:	learn: 0.2117338	total: 5.37s	remaining: 5s
518:	learn: 0.2117321	total: 5.38s	remaining: 4.99s
519:	learn: 0.2117309	total: 5.39s	remaining: 4.97s
520:	learn: 0.2117126	total: 5.4s	remaining: 4.96s
521:	learn: 0.2117110	total: 5.41s	remaining: 4.95s
522:	learn: 0.2117096	total: 5.41s	remaining: 4.94s
523:	learn: 0.2117084	total: 5.42s	remaining: 4.92s
524:	learn: 0.2117079	total: 5.43s	remaining: 4.91s
525:	learn: 0.2117051	total: 5.44s	remaining: 4.9s
526:	learn: 0.2117036	total: 5.45s	remaining: 4.89s
527:	learn: 0.2117027	total: 5.46s	remaining: 4.88s
528:	learn: 0.2117006	total: 5.46s	remaining: 4.86s
529:	learn: 0.2116889	total: 5.47s	remaining: 4.85s
530:	learn: 0.2116856	total: 5.48s	remaining: 4.84s
531:	learn: 0.211

677:	learn: 0.2105324	total: 6.73s	remaining: 3.2s
678:	learn: 0.2105320	total: 6.75s	remaining: 3.19s
679:	learn: 0.2105287	total: 6.75s	remaining: 3.18s
680:	learn: 0.2105251	total: 6.76s	remaining: 3.17s
681:	learn: 0.2105164	total: 6.77s	remaining: 3.16s
682:	learn: 0.2105107	total: 6.78s	remaining: 3.15s
683:	learn: 0.2105087	total: 6.79s	remaining: 3.14s
684:	learn: 0.2105081	total: 6.8s	remaining: 3.13s
685:	learn: 0.2105051	total: 6.81s	remaining: 3.12s
686:	learn: 0.2104933	total: 6.81s	remaining: 3.1s
687:	learn: 0.2104863	total: 6.82s	remaining: 3.09s
688:	learn: 0.2104821	total: 6.83s	remaining: 3.08s
689:	learn: 0.2104774	total: 6.84s	remaining: 3.07s
690:	learn: 0.2104478	total: 6.85s	remaining: 3.06s
691:	learn: 0.2104301	total: 6.86s	remaining: 3.05s
692:	learn: 0.2104157	total: 6.86s	remaining: 3.04s
693:	learn: 0.2104068	total: 6.87s	remaining: 3.03s
694:	learn: 0.2104056	total: 6.88s	remaining: 3.02s
695:	learn: 0.2104026	total: 6.89s	remaining: 3.01s
696:	learn: 0.2

857:	learn: 0.2092152	total: 8.34s	remaining: 1.38s
858:	learn: 0.2091879	total: 8.36s	remaining: 1.37s
859:	learn: 0.2091875	total: 8.36s	remaining: 1.36s
860:	learn: 0.2091831	total: 8.37s	remaining: 1.35s
861:	learn: 0.2091830	total: 8.38s	remaining: 1.34s
862:	learn: 0.2091827	total: 8.39s	remaining: 1.33s
863:	learn: 0.2091826	total: 8.4s	remaining: 1.32s
864:	learn: 0.2091813	total: 8.4s	remaining: 1.31s
865:	learn: 0.2091699	total: 8.41s	remaining: 1.3s
866:	learn: 0.2091693	total: 8.42s	remaining: 1.29s
867:	learn: 0.2091435	total: 8.43s	remaining: 1.28s
868:	learn: 0.2091232	total: 8.44s	remaining: 1.27s
869:	learn: 0.2091117	total: 8.45s	remaining: 1.26s
870:	learn: 0.2091103	total: 8.45s	remaining: 1.25s
871:	learn: 0.2091101	total: 8.46s	remaining: 1.24s
872:	learn: 0.2090903	total: 8.47s	remaining: 1.23s
873:	learn: 0.2090878	total: 8.48s	remaining: 1.22s
874:	learn: 0.2090876	total: 8.49s	remaining: 1.21s
875:	learn: 0.2090712	total: 8.5s	remaining: 1.2s
876:	learn: 0.209

In [197]:
INPUT = "dataset_40163_1.txt"

In [198]:
results = ""
with open(INPUT, 'r') as fin:
    for line in notebook.tqdm(fin):
        words = line.split(' ')
        current_index = 0
        line_result = ""
        word_pos = 0
        for word in words:
            pure_word = regex.sub('', word).replace('ё', 'e')
            if pure_word:
                #ftrs = enc.transform([build_features(pure_word, word_pos, len(words))]).toarray()[0]
                ftrs = build_features(pure_word, word_pos, len(words))
                word_pos += 1
                #print(ftrs)
                label = clf.predict([ftrs])[0][0]
                #print(label)
                if label != "OUT":
                    line_result += str(current_index) + " " + str(len(pure_word)) + " " + label + " "
            current_index += len(word) + 1
        results += line_result + "EOL\n"
            

In [199]:
OUTPUT = "output.txt"

In [200]:
with open(OUTPUT, 'w') as fout:
    fout.write(results)

In [201]:
res = ""
with open(INPUT, 'r') as fin:
    i = 0
    for line in notebook.tqdm(fin):
        words = line.split(' ')
        current_index = 0
        line_result = ""
        for word in words:
            current_index += len(word)
            line_result += word + "(" + str(current_index) + "/" + str(len(word)) + ") "
            current_index += 1
        res += str(i) + ") "+ line_result + "\n\n"
        i += 1
            

In [202]:
with open("len.txt", 'w') as fout:
    fout.write(res)

In [203]:
from sklearn.metrics import f1_score

def accuracy(pred, y):
    _, ansx = pred.max(dim=1)
    ansx = ansx.cpu().numpy().reshape(-1)
    y_ = y.cpu().numpy()
    return f1_score(ansx, y_, average='macro')

In [206]:

import torch.nn as nn

In [207]:
class MyLSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers, bidirectional, vocab_size, tagset_size):
        super(MyLSTM, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, batch_first=True)
        # self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, tagset_size)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, batch):
        embeds = self.word_embeddings(batch)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.fc2(self.dropout(lstm_out))
        tag_scores = F.log_softmax(tag_space, dim=2)
        return tag_scores

In [247]:
def fit(model, train, val, optimizer, loss_function, epoch_cnt, batch_size):
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size)
    train_loss_values = []
    val_loss_values = []
            
    for epoch in range(epoch_cnt):
        for batch_data in train_loader:
            x, y = batch_data[0].to(device), batch_data[1].to(device)
            optimizer.zero_grad()
            output = model(x.long())
            output = output.view(-1, 3)
            y = y.reshape(-1)
            loss = loss_function(output, y.long())
            train_loss_values.append(loss)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            acc = accuracy(output, y)

        with torch.no_grad():
            loss_values = []
            acc_values = []
            for batch_data in val_loader:
                x, y = batch_data[:, 0].to(device), batch_data[:, 1].to(device)
                output = model(x.long())
                output = output.view(-1, 3)
                y = y.reshape(-1)
                loss = loss_function(output, y.long())
                acc = accuracy(output, y)
                loss_values.append(loss.item())
                acc_values.append(acc)
            val_loss_values.append(np.mean(np.array(loss_values)))

    return train_loss_values, val_loss_values

In [248]:
embedding_dim = 64
hidden_dim = 128
num_layers = 1
bidirectional = False
vocab_size = len(word_to_idx) + 3
tagset_size = 3
epoch_cnt = 320
batch_size = 256

In [249]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [250]:
model = MyLSTM(embedding_dim, hidden_dim, num_layers, bidirectional, vocab_size, tagset_size)
model = model.float()
model = model.to(device)

In [251]:
import torch.optim as optim
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)

In [252]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(features, test_size=0.1, random_state=42)

In [255]:
train_loss_values, val_loss_values =\
    fit(model, train, val, optimizer, loss_function, epoch_cnt, batch_size)

In [257]:
with torch.no_grad():
    test_loader = torch.utils.data.DataLoader(test_padded, batch_size=batch_size)
    ans = None
    
    for batch_data in test_loader:
        x = batch_data.to(device)
        output = model(x.long())
        _, ansx = output.max(dim=2)
        ansx = ansx.cpu().numpy()
        if ans is None:
            ans = ansx
        else:
            ans = np.append(ans, ansx, axis=0)

In [259]:
output_filename = "data/output.txt"
int_to_tag = {1:"PERSON" , 2:"ORG"}
with open(output_filename, "w", encoding="utf-8") as output_file:
    for sent, pos, tags in zip(test_padded, test_pos, ans):
        for i in range(len(pos)):
            if tags[i] == 1 or tags[i] == 2:
                output_file.write("%d %d %s " % (pos[i][0], pos[i][1], int_to_tag[tags[i]]))
        output_file.write("EOL\n")